In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from json import loads, dumps
import uuid
from datetime import datetime
NotebookExecutionId = str(uuid.uuid4())

In [ ]:
def format_guid(input_str: str) -> str:
    """
    Formats the input string by adding hyphens at specific positions if they are missing.
    Parameters:
    - input_str (str): The input string to be formatted.
    Returns:
    - str: The formatted string.
    """
    if "-" not in input_str and len(input_str) == 32:
        formatted_str = '-'.join([input_str[:8], input_str[8:12], input_str[12:16], input_str[16:20], input_str[20:]])
        return formatted_str
    else:
        return input_str
def is_valid_guid(guid_str: str) -> bool:
    """
    Check if a string is a valid GUID.

    Parameters:
    - guid_str (str): The string to be checked.

    Returns:
    - bool: True if the string is a valid GUID, False otherwise.
    """
    try:
        uuid_obj = uuid.UUID(guid_str)
        # Check if the UUID is valid
        return str(uuid_obj) == guid_str
    except ValueError:
        return False

In [ ]:
Path = ""
PipelineProcessGuid = ""
PipelineRunGuid = ""
PipelineGuid = ""
TriggerGuid = ""
TriggerTime = ""
TriggerType = ""
useRootDefaultLakehouse= True

In [ ]:
## always use notebook guid instead of pipeline id
PipelineName = mssparkutils.runtime.context.get('currentNotebookName')
PipelineGuid = str(mssparkutils.runtime.context.get('currentNotebookId'))
WorkspaceGuid = mssparkutils.runtime.context.get('currentWorkspaceId')
PipelineParentRunGuid = mssparkutils.runtime.context.get('PipelineParentRunGuid')
PipelineRunGuid = str(uuid.uuid4())
TriggerGuid = format_guid(TriggerGuid)

if not PipelineParentRunGuid:
    PipelineParentRunGuid='00000000-0000-0000-0000-000000000000'
if not PipelineProcessGuid:
    PipelineProcessGuid='00000000-0000-0000-0000-000000000000'   
if not PipelineGuid:
    PipelineGuid='00000000-0000-0000-0000-000000000000'
if not TriggerGuid or not is_valid_guid(TriggerGuid):
    TriggerGuid = '00000000-0000-0000-0000-000000000000'
if not TriggerTime:
    TriggerTime=''
if not TriggerType:
    TriggerType=''
if not WorkspaceGuid:
    WorkspaceGuid='00000000-0000-0000-0000-000000000000'

## always use notebook guid instead of pipeline id
#PipelineName = mssparkutils.runtime.contex.Notebookname
#PipelineGuid = mssparkutils.runtime.context.currentNotebookId

starttime = datetime.now()

In [ ]:
# Iterate through each object and add items to the "params" dictionary
if isinstance(Path, str):
    path_data = loads(Path)
elif isinstance(Path, List):
    path_data = Path
elif isinstance(Path, dict):
    path_data = [Path]


for i, item in enumerate( path_data):
    item["notebook_path"] = item['path']
    data_source = item['params']['target_schema'].split('/')[0]
    schema_table = "".join(item['params']['target_name'].split('_')[:2])
    item["notebook_activity_id"] = f"{data_source}_{schema_table}"
    item["params"]["PipelineGuid"] = PipelineGuid
    item["params"]["TriggerGuid"] = TriggerGuid
    item["params"]["TriggerType"] = TriggerType
    item["params"]["TriggerTime"] = TriggerTime
    item["params"]["WorkspaceGuid"] = WorkspaceGuid  
    item["params"]["PipelineParentRunGuid"] = PipelineParentRunGuid
    item["params"]["NotebookExecutionId"] = NotebookExecutionId
    item["params"]["useRootDefaultLakehouse"] = useRootDefaultLakehouse

# path_data = dumps(path_data, indent=2)

notebooks = path_data

#print(dumps(notebooks, indent=2))


In [ ]:
activities = [{"name": nb['notebook_activity_id'], # activity name, must be unique
               "path": nb['notebook_path'], # notebook path
               "timeoutPerCellInSeconds": 600, # max timeout for each cell, default to 90 seconds
               "args": nb["params"],
               #"workspace": "workspace1", # workspace name, default to current workspace
               "retry": 1, # max retry times, default to 0
               "retryIntervalInSeconds": 0} for nb in notebooks]
cmd_dag = {"activities": activities,
           "timeoutInSeconds": 7200, # max timeout for the entire pipeline, default to 12 hours now set to 2 hrs
           "concurrency": 0} # max number of notebooks to run concurrently, default to unlimited
#print(dumps(cmd_dag, indent=2))

## Execute Notebooks

In [ ]:
#Execute Notebooks
results  = mssparkutils.notebook.runMultiple(cmd_dag)